# Obligatorio 2 - Big Data Science

Integrantes del grupo: [completar aquí los nombres de los integrantes]

Debajo de cada pregunta o tarea incluya las celdas necesarias para desarrolar la respuesta. Puede usar una o varias celdas de código o mark down (https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

Para entregar, renombrar este notebook como "Obligatorio 2 - Apellido1 - Apellido 2 - Apellido 3" con los apellidos de los miembros del grupo. Un solo integrante del grupo debe realizar la entrega. 


1. Cargar los datos del Obligatorio 1, de entrenamiento (.data) y validación (.test) en spark dataframes (distintos). Los nombres de las columnas deben corresponder a los especificados en "Attribute Information".

In [1]:
from pyspark.context import SparkContext
sc = SparkContext('local', 'test')

In [2]:
sc

<SparkContext master=local appName=test>

In [4]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

In [5]:
spark

In [6]:
from pyspark.sql.types import *

# Schema de los dataset
schema = StructType([
    StructField(name='age', dataType=IntegerType(), nullable=False),
    StructField(name='workclass', dataType=StringType(), nullable=False),
    StructField(name='fnlwgt', dataType=StringType(), nullable=False), # Se pone en tipo string porque al ponerlo en tipo Integer o Long carga la columna con valores null
    StructField(name='education', dataType=StringType(), nullable=False),
    StructField(name='education_num', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='marital-status', dataType=StringType(), nullable=False),
    StructField(name='occupation', dataType=StringType(), nullable=False),
    StructField(name='relationship', dataType=StringType(), nullable=False),
    StructField(name='race', dataType=StringType(), nullable=False),
    StructField(name='sex', dataType=StringType(), nullable=False),
    StructField(name='capital_gain', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='capital_loss', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='hours_per_week', dataType=StringType(), nullable=False), # Mismo problema que en la columna 'fnlwgt'
    StructField(name='native_country', dataType=StringType(), nullable=False),
    StructField(name='income', dataType=StringType(), nullable=False)
])

In [7]:
df_adult = spark.read.csv("adult.data", header=True, schema=schema)
df_test = spark.read.csv("adult.test", header=True, schema=schema)

In [8]:
df_adult.show(10)

+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass| fnlwgt|    education|education_num|      marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+-------+-------------+-------------+--------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 50| Self-emp-not-inc|  83311|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|           0|           0|            13| United-States| <=50K|
| 38|          Private| 215646|      HS-grad|            9|            Divorced| Handlers-cleaners| Not-in-family| White|   Male|           0|           0|            40| United-States| <=50K|
| 53|          Private| 234721|    

In [9]:
df_test.show(10)

+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+-------+
|age|        workclass| fnlwgt|    education|education_num|     marital-status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country| income|
+---+-----------------+-------+-------------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+-------+
| 25|          Private| 226802|         11th|            7|      Never-married| Machine-op-inspct|     Own-child| Black|   Male|           0|           0|            40| United-States| <=50K.|
| 38|          Private|  89814|      HS-grad|            9| Married-civ-spouse|   Farming-fishing|       Husband| White|   Male|           0|           0|            50| United-States| <=50K.|
| 28|        Local-gov| 336951|   A

2. Seleccionar un conjunto relevante de 5 atributos y crear un Spark Pipeline en el que el estimator sea un DecisionTreeClassifier (https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Puede utilizar libremente los transformers/estimators de Spark para realizar ingeniería de atributos (StringIndexer, OneHotEncoding, etc).

In [19]:
# atributos = ["age", "sex", "occupation", "education", "workclass"]

#Primero hay que bucketizar la edad. La separamos en 3 buckets. 

def calculo_de_buckets(df, cantidad_buckets, columna):
    df.createOrReplaceTempView("df_view")
    instancias_totales = spark.sql('''SELECT count(*) as count FROM df_view''').first()['count']
    instancias_en_bucket = instancias_totales / cantidad_buckets
    
    limites = []
    ultimo_limite_superior = None
    for limit in range(0, instancias_totales - 1, round(instancias_en_bucket)): # se pone un -1 porque instancias totales es el count pero queremos llegar hasta el indice        
        query = spark.sql(f'''SELECT {columna} FROM df_view ORDER BY 1 asc''').collect()
        limite_inferior = query[limit][columna]
        
        if(ultimo_limite_superior is not None):
            if ultimo_limite_superior == limite_inferior:
                limite_inferior += 1;
        
        limite = [limite_inferior, query[limit + round(instancias_en_bucket)][columna]]
        limites.append(limite)
        
        ultimo_limite_superior = query[limit + round(instancias_en_bucket)][columna]

    return limites

limites = calculo_de_buckets(df_adult, 3, "age")

[[17, 31], [32, 44], [45, 90]]


In [38]:
from pyspark.sql import functions as F

def age_bucket_udf(age):
    indexes = range(len(limites))
    for index, limite in zip(indexes, limites):
        if age in range(limite[0], limite[1] + 1):    #Se agrega el +1 para hacer el range inclusivo superiormente
            return index
    return -1


df_adult_bucketized = df_adult.withColumn("age_bucket", age_udf(F.col("age")))
df_adult_bucketized.show(10)

Py4JJavaError: An error occurred while calling o336.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 94.0 failed 1 times, most recent failure: Lost task 0.0 in stage 94.0 (TID 1952) (DESKTOP-6G3A0KP executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:70)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anonfun$ma

3. Hacer el fit del pipeline con los datos de entrenamiento. 


In [5]:
#

4. Graficar la curva ROC utilizando los datos de validación (sin usar el paquete de evluación de Spark pyspark.ml.evaluation)

In [6]:
#

5. Escribir dos funciones train() y predict() que creen el arbol de acuerdo a la metodología vista en clase (utilizando entropía como métrica de homogeneidad de clases).
```
def train(train_dataframe):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
     pass        
```

```
def predict(tree, train_dataframe)
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    pass
```    
Ejemplo de uso:

```
tree = train(train_dataframe)
predictions_df = predict(tree, train_dataframe)
```

In [7]:
# 

<i>[Extra credit y alumnos masters]</i>
    
5.  Mejorar la implementación de su algoritmo evitando hacer el split cuando no se logra un mínimo de Information Gain ( https://en.wikipedia.org/wiki/Information_gain_in_decision_trees   )


In [8]:
#